In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

import os
from pathlib import Path

from vivarium_helpers.utils import convert_to_categorical, constant_categorical, print_memory_usage, Timer

from vivarium_helpers.projects.alzheimers import loading, population
from vivarium_helpers.projects.alzheimers.population import RunType
from vivarium_helpers.projects.alzheimers.results import AlzheimersResultsProcessor

!date
!whoami
!pwd

Mon Nov 24 21:04:32 PST 2025
ndbs
/mnt/share/code/ndbs/vivarium_research_alzheimers/results_tables


# Record version of Vivarium Helpers

In [3]:
!conda list | grep vivarium

# packages in environment at /ihme/homes/ndbs/miniconda3/envs/vivarium_csu_alzheimers_artifact:
vivarium                  3.4.12                   pypi_0    pypi
vivarium-build-utils      2.0.4                    pypi_0    pypi
vivarium-cluster-tools    2.1.17                   pypi_0    pypi
vivarium-csu-alzheimers   0.1.dev18+g43e997c.d20250808          pypi_0    pypi
vivarium-dependencies     1.0.1                    pypi_0    pypi
vivarium-gbd-access       4.2.4                    pypi_0    pypi
vivarium-helpers          0.2.0                     dev_0    <develop>
vivarium-inputs           6.0.4                    pypi_0    pypi
vivarium-public-health    4.3.5                    pypi_0    pypi


In [4]:
!pip freeze | grep vivarium

vivarium==3.4.12
vivarium_build_utils==2.0.4
vivarium_cluster_tools==2.1.17
-e git+https://github.com/ihmeuw/vivarium_csu_alzheimers.git@263eb3d9b80b13735a7380bb892430eaba92603a#egg=vivarium_csu_alzheimers
vivarium_dependencies==1.0.1
vivarium_gbd_access==4.2.4
-e git+https://github.com/ihmeuw/vivarium_helpers.git@dbd5d0258ecd95eb5b409eee9a5322e4830b1d44#egg=vivarium_helpers
vivarium_inputs==6.0.4
vivarium_public_health==4.3.5


# Define data directories

In [5]:
# Project directory
project_dir = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/')

# Output directory to store final results for client
output_dir = Path(r"J:\Project\simulation_science\alzheimers\results_in_progress".replace('\\', '/').replace('J:', '/snfs1'))
print(output_dir.exists())

# Output from multistate life table
mslt_output_dir = output_dir.parent / 'results_11_14_2025_mslt'
print(mslt_output_dir.exists())
output_dir

True
True


PosixPath('/snfs1/Project/simulation_science/alzheimers/results_in_progress')

In [6]:
# # For testing: Run directory containing model 8.3 results for all
# # locations
# model_run_subdir = 'results/abie_consistent_model_test/united_states_of_america/2025_10_28_08_55_05/'

# # Results directory for model 8.3, for testing
# results_dirs = project_dir / model_run_subdir / 'results/'

# Artifact for models 8.3 - 8.7
artifact_model_number = '8.3'


In [7]:
# Model 8.4 results (final runs for 10/31/25 intermediate results)
# Each batch run contains all locations, 100 random seeds, and 3 or 4 draws
model_run_dir_8_4 = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec')

# Model 8.7 results (updated final run completed on 11/10/2025)
model_run_dir_8_7 = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec')

# Batch runs are stored in "timestamp" subdirectories of the model run
# directory
# with os.scandir(model_run_dir_8_7) as entries:
#     batch_run_dirs_8_7 = [entry.path for entry in entries]
batch_run_dirs_8_7 = loading.list_paths(model_run_dir_8_7)

# Select a model and filter for testing
batch_run_dirs = batch_run_dirs_8_7#[:2]

batch_run_dirs

['/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_06_06_42_37',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_07_09_05_46',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_10_07_44_18',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_06_06_43_53',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_06_13_00_54',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_16_28_26',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_07_13_14_27',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/result

# Create an AlzheimersResultsProcessor object to process results

In [8]:
results = AlzheimersResultsProcessor('8.3', RunType.FINAL, batch_run_dirs=batch_run_dirs)

In [9]:
results.initial_simulation_population

2000000

In [10]:
results.locations

['United States of America',
 'Brazil',
 'China',
 'Germany',
 'Israel',
 'Japan',
 'Spain',
 'Sweden',
 'Taiwan (Province of China)',
 'United Kingdom']

In [11]:
results.draws

[457,
 169,
 323,
 258,
 446,
 89,
 41,
 113,
 356,
 273,
 158,
 24,
 265,
 317,
 177,
 199,
 392,
 346,
 480,
 262,
 460,
 499,
 2,
 13,
 46]

# Load population data and calculate model scale

In [12]:
with Timer():
    results.load_population_data(append_aggregates=True)
results.ops.describe(results.model_scale)

Elapsed time: 0:00:17.409072


,count,mean,std,min,25%,50%,75%,max
location,,,,,,,,
Brazil,25.0,1.116473,0.102910,0.886671,1.059915,1.119741,1.145108,1.386433
China,25.0,0.123020,0.007061,0.106308,0.120095,0.123499,0.125744,0.143724
Germany,25.0,0.922878,0.055810,0.854854,0.876986,0.914862,0.983316,1.012529
Israel,25.0,41.883212,2.979755,37.896818,39.739917,40.651231,43.913069,48.068295
Japan,25.0,0.586225,0.042318,0.511002,0.557443,0.585295,0.611764,0.694869
Spain,25.0,3.192170,0.169602,2.875882,3.073433,3.190730,3.289187,3.574185
Sweden,25.0,12.993787,0.944507,11.361366,12.398862,12.890094,13.665418,14.830542
Taiwan (Province of China),25.0,8.853801,0.468546,8.050507,8.487516,8.855923,9.154749,9.783891
United Kingdom,25.0,2.057251,0.093390,1.891726,1.987250,2.042450,2.134812,2.215977


In [13]:
results.person_time

,location,sex,event_year,input_draw,value,age_group
0,United States of America,Female,2025,457,1.091295e+07,60_to_64
1,United States of America,Female,2025,169,1.123373e+07,60_to_64
2,United States of America,Female,2025,323,1.021452e+07,60_to_64
3,United States of America,Female,2025,258,1.064787e+07,60_to_64
4,United States of America,Female,2025,446,1.064298e+07,60_to_64
...,...,...,...,...,...,...
911995,Spain,Both,2100,499,1.730644e+07,60_plus
911996,Sweden,Both,2100,499,3.703980e+06,60_plus
911997,Taiwan (Province of China),Both,2100,499,9.136561e+06,60_plus
911998,United Kingdom,Both,2100,499,2.273886e+07,60_plus


# Deaths and averted deaths

In [14]:
with Timer():
    results.load_measure_from_batch_runs('deaths', filters=loading.FINAL_RESULTS_FILTERS['deaths'])

print(len(results.data.deaths), 'rows')
results.data.deaths.head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025'), ('entity', '=', 'alzheimers_disease_state')]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,deaths,baseline,Female,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,deaths,baseline,Male,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,deaths,bbbm_testing,Female,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,deaths,bbbm_testing,Male,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,deaths,bbbm_testing_and_treatment,Female,0.0,Japan


In [15]:
deaths_prepped = results.process_deaths(results.data.deaths)
deaths_prepped

47.885504 MB measure
15.965504 MB minuend
31.925504 MB subtrahend
10.269531 MB minuend re-indexed
20.529531 MB subtrahend re-indexed
20.529591 MB difference
36.485636 MB difference with reset index
37.625961 MB final difference


,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,value,location,metric,disease_stage
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,Deaths Associated with AD,baseline,Female,0.0,Japan,Number,alzheimers_disease_state
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,Deaths Associated with AD,baseline,Male,0.0,Japan,Number,alzheimers_disease_state
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,Deaths Associated with AD,bbbm_testing,Female,0.0,Japan,Number,alzheimers_disease_state
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,Deaths Associated with AD,bbbm_testing,Male,0.0,Japan,Number,alzheimers_disease_state
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,Deaths Associated with AD,bbbm_testing_and_treatment,Female,0.0,Japan,Number,alzheimers_disease_state
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,13,Averted Deaths Associated with AD,bbbm_testing_and_treatment,Male,-383.0,Brazil,Number,alzheimers_disease_state
2849996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,46,Averted Deaths Associated with AD,bbbm_testing,Female,0.0,Brazil,Number,alzheimers_disease_state
2849997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,46,Averted Deaths Associated with AD,bbbm_testing,Male,0.0,Brazil,Number,alzheimers_disease_state
2849998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,46,Averted Deaths Associated with AD,bbbm_testing_and_treatment,Female,-636.0,Brazil,Number,alzheimers_disease_state


In [16]:
deaths_output = results.summarize_and_beautify(deaths_prepped, 'entity')
deaths_output

Timer SummarizingTimer started
2025-11-24 21:06:00.789585
2025-11-24 21:06:40.096238
2025-11-24 21:15:30.987117
Elapsed time: 570.2117 seconds


,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,60_to_64,Female,AD Dementia,Reference,Deaths Associated with AD,Number,865.570989,664.279568,1108.070345
1,2025,Brazil,60_to_64,Male,AD Dementia,Reference,Deaths Associated with AD,Number,366.099864,292.105112,469.009694
2,2025,Brazil,60_to_64,Both,AD Dementia,Reference,Deaths Associated with AD,Number,1231.670854,964.563858,1577.080038
3,2025,Brazil,60_to_64,Female,AD Dementia,BBBM Testing Only,Deaths Associated with AD,Number,865.570989,664.279568,1108.070345
4,2025,Brazil,60_to_64,Male,AD Dementia,BBBM Testing Only,Deaths Associated with AD,Number,366.099864,292.105112,469.009694
...,...,...,...,...,...,...,...,...,...,...,...
364795,2100,United States of America,60_plus,Male,AD Dementia,BBBM Testing Only,Averted Deaths Associated with AD,Rate,0.000000,0.000000,0.000000
364796,2100,United States of America,60_plus,Both,AD Dementia,BBBM Testing Only,Averted Deaths Associated with AD,Rate,0.000000,0.000000,0.000000
364797,2100,United States of America,60_plus,Female,AD Dementia,BBBM Testing and Treatment,Averted Deaths Associated with AD,Rate,0.000053,0.000035,0.000069
364798,2100,United States of America,60_plus,Male,AD Dementia,BBBM Testing and Treatment,Averted Deaths Associated with AD,Rate,0.000029,0.000019,0.000037


In [17]:
deaths_output.to_csv(output_dir / "deaths.csv", index=False)

# Prevalence

In [18]:
filename = 'person_time_alzheimers_disease_and_other_dementias'
short_name = 'person_time_ad'

with Timer():
    results.load_measure_from_batch_runs(filename, short_name, filters=loading.FINAL_RESULTS_FILTERS[filename])

print(len(results.data[short_name]), 'rows')
results.data[short_name].head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025')]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/spain.hdf', 'Sweden': '/m

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,sub_entity,treatment,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,person_time,baseline,Female,alzheimers_blood_based_biomarker_state,susceptible_to_treatment,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,person_time,baseline,Female,alzheimers_blood_based_biomarker_state,waiting_for_treatment,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,person_time,baseline,Female,alzheimers_blood_based_biomarker_state,full_effect_long,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,person_time,baseline,Female,alzheimers_blood_based_biomarker_state,full_effect_short,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,169,person_time,baseline,Female,alzheimers_blood_based_biomarker_state,waning_effect_long,0.0,Japan


In [19]:
prevalence_counts = results.process_prevalence_counts(results.data.person_time_ad)
print_memory_usage(prevalence_counts)
print(len(prevalence_counts), 'rows')
prevalence_counts.tail()

107.73999 MB 
5130000 rows


,age_group,artifact_path,entity,entity_type,event_year,input_draw,location,measure,scenario,sex,sub_entity,value,metric,disease_stage
5129995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2100,499,Brazil,Prevalence,bbbm_testing_and_treatment,Female,alzheimers_mild_cognitive_impairment_state,90833.571526,Number,alzheimers_mild_cognitive_impairment_state
5129996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2100,499,Brazil,Prevalence,bbbm_testing_and_treatment,Female,alzheimers_disease_state,206003.071869,Number,alzheimers_disease_state
5129997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2100,499,Brazil,Prevalence,bbbm_testing_and_treatment,Male,alzheimers_blood_based_biomarker_state,59830.039699,Number,alzheimers_blood_based_biomarker_state
5129998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2100,499,Brazil,Prevalence,bbbm_testing_and_treatment,Male,alzheimers_mild_cognitive_impairment_state,46955.252567,Number,alzheimers_mild_cognitive_impairment_state
5129999,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2100,499,Brazil,Prevalence,bbbm_testing_and_treatment,Male,alzheimers_disease_state,91416.071184,Number,alzheimers_disease_state


In [20]:
with Timer():
    prevalence_output = results.summarize_and_beautify(prevalence_counts)
print_memory_usage(prevalence_output)
print(len(prevalence_output), 'rows')
prevalence_output.tail()

Timer SummarizingTimer started
2025-11-24 21:45:54.200330


2025-11-24 21:46:41.697041
2025-11-24 22:02:09.790635
Elapsed time: 975.6061 seconds
Elapsed time: 0:16:15.606325
21.01924 MB 
656640 rows


,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
656635,2100,United States of America,60_plus,Male,AD Dementia,BBBM Testing Only,Prevalence,Rate,0.019652,0.017796,0.021055
656636,2100,United States of America,60_plus,Both,AD Dementia,BBBM Testing Only,Prevalence,Rate,0.028895,0.026629,0.031322
656637,2100,United States of America,60_plus,Female,AD Dementia,BBBM Testing and Treatment,Prevalence,Rate,0.036068,0.033293,0.039431
656638,2100,United States of America,60_plus,Male,AD Dementia,BBBM Testing and Treatment,Prevalence,Rate,0.019284,0.017501,0.020691
656639,2100,United States of America,60_plus,Both,AD Dementia,BBBM Testing and Treatment,Prevalence,Rate,0.028368,0.026153,0.030766


In [21]:
# Save prevalence data
prevalence_output.to_csv(output_dir / 'prevalence.csv', index=False)

In [22]:
!date

Mon Nov 24 22:02:16 PST 2025
